In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB, GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
FEATURES_SET = {
    "feature": 1,
    "permission": 2,
    "activity": 3,
    "service_receiver": 3,
    "provider": 3,
    "service": 3,
    "intent": 4,
    "api_call": 5,
    "real_permission": 6,
    "call": 7,
    "url": 8
}


def count_feature_set(lines):
    """
    Count how many features belong to a specific set
    :param lines: features in the text file
    :return:
    """
    features_map = {x: 0 for x in range(1, 9)}
    for l in lines:
        if l != "\n":
            set = l.split("::")[0]
            features_map[FEATURES_SET[set]] += 1
    features = []
    for i in range(1, 9):
        features.append(features_map[i])
    return features


In [3]:
def read(LOAD_DATA=False):
    if LOAD_DATA:
        print("Previous data not loaded. Attempt to read data ...")
        mypath = r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\feature_vectors\feature_vectors"
        onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

        print("Reading csv file for ground truth ...")
        ground_truth = np.loadtxt(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\sha256_family.csv", delimiter=",", skiprows=1, dtype=str)
        # print ground_truth.shape
        # families = np.unique(ground_truth[:, 1])
        # print families
        # print len(families)

        print("Reading positive and negative texts ...")
        pos = []
        neg = []
        for virus in tqdm(onlyfiles):
            if virus in ground_truth[:, 0]:
                pos.append(virus)
            else:
                if len(neg) < 5560:
                #if len(neg) < 22240:
                    neg.append(virus)

        print("Extracting features ...")
        x = []
        y = []
        for text_file in tqdm(pos):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(1)

        for text_file in tqdm(neg):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(0)

        print("Data is read successfully:")
        x = np.array(x)
        y = np.array(y)
        print(x.shape, y.shape)

        print("Saving data under data_numpy directory ...")
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy", x)
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy", y)

        return x, y
    else:
        print("Loading previous data ...")
        x_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
        y_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")
        print(x_.shape, y_.shape)
        # print x == x_, y == y_
        return x_, y_


def map_family_to_category(families):
    out = {}
    count = 1
    for family in families:
        out[family] = count
        count += 1
    return out


if __name__ == "__main__":
    x, y = read(LOAD_DATA=True)

Previous data not loaded. Attempt to read data ...
Reading csv file for ground truth ...
Reading positive and negative texts ...


100%|██████████| 129013/129013 [00:01<00:00, 70352.57it/s]


Extracting features ...


100%|██████████| 5560/5560 [00:28<00:00, 192.88it/s]

Data is read successfully:
(11120, 8) (11120,)
Saving data under data_numpy directory ...


In [4]:
x_all, y_all = read(LOAD_DATA=False)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.33, stratify=y_all)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Loading previous data ...
(11120, 8) (11120,)
(7450, 8) (7450,)
(3670, 8) (3670,)


In [5]:
x_all[0]

array([ 2, 11,  5,  3,  7,  6, 11, 26])

In [6]:
# Criando e treinando o modelo SVM
model = svm.SVC(kernel='linear', C=0.0001, verbose=1)
model.fit(x_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = model.predict(x_test)

# Avaliando a precisão do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo SVM:", accuracy)

# Calculando a matriz de confusão
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(conf_matrix)

# Calculando recall, precisão e f1-score
recall = metrics.recall_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred)
print("Recall:", recall)
print("Precisão:", precision)
print("F1-Score:", f1)

# Calculando a curva ROC e AUC
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
auc = metrics.auc(fpr, tpr)
print("AUC Score:", auc)

# Calculando o índice de Matthews
mcc = metrics.matthews_corrcoef(y_test, y_pred)
print("Índice de Matthews:", mcc)

[LibSVM]Acurácia do modelo SVM: 0.808174386920981
Matriz de Confusão:
[[1610  225]
 [ 479 1356]]
Recall: 0.7389645776566758
Precisão: 0.857685009487666
F1-Score: 0.7939110070257611
AUC Score: 0.8081743869209809
Índice de Matthews: 0.6223396194555193
